# Gold Layer - Data Serving with SQL & MLlib

## Lending Club Loan Data Pipeline

**Use Case:** Predict loan default risk and analyze factors affecting loan approval

This notebook implements the Gold (Serving) layer of the Medallion Architecture:
- Business analytics using **Spark SQL**
- Machine Learning using **MLlib** (Loan Default Prediction)
- Create aggregated tables ready for dashboards and applications

**Note:** Unlike the Silver layer, this notebook uses high-level APIs (DataFrames, SQL, MLlib) as permitted by the project requirements.

## 1. Setup and Configuration

In [1]:
import os
import time
import numpy as np
from datetime import datetime

import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

# ML imports
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    VectorAssembler, 
    StringIndexer, 
    OneHotEncoder,
    StandardScaler,
    Imputer
)
from pyspark.ml.classification import (
    LogisticRegression, 
    RandomForestClassifier,
    GBTClassifier
)
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator,
    MulticlassClassificationEvaluator
)
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

print("Imports complete!")

Imports complete!


In [2]:
# Initialize Spark Session
spark = SparkSession.builder \
        .appName("LendingClub-Gold-Layer") \
        .master("spark://spark-master:7077") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.executor.memory", "4g") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.cores", "4") \
        .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

print(f"Spark Version: {spark.version}")
print(f"Spark UI available at: {sc.uiWebUrl}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/27 12:44:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Version: 3.5.0
Spark UI available at: http://spark-master:4040


In [3]:
# Define paths
SILVER_PATH = "../data/medallion/silver/"
GOLD_PATH = "../data/medallion/gold/"

SILVER_ACCEPTED_PATH = os.path.join(SILVER_PATH, "accepted_loans")
SILVER_REJECTED_PATH = os.path.join(SILVER_PATH, "rejected_loans")

# Create gold directory
os.makedirs(GOLD_PATH, exist_ok=True)

print(f"Silver input path: {SILVER_PATH}")
print(f"Gold output path: {GOLD_PATH}")

Silver input path: ../data/medallion/silver/
Gold output path: ../data/medallion/gold/


## 2. Load Silver Data

In [4]:
# Load cleaned data from Silver layer
loans_df = spark.read.parquet(SILVER_ACCEPTED_PATH)
rejected_df = spark.read.parquet(SILVER_REJECTED_PATH)

print(f"Accepted loans: {loans_df.count():,} rows")
print(f"Rejected loans: {rejected_df.count():,} rows")
print(f"\nAccepted loans columns: {len(loans_df.columns)}")

Accepted loans: 2,231,965 rows
Rejected loans: 20,116,218 rows

Accepted loans columns: 30


In [5]:
# Show schema
loans_df.printSchema()

root
 |-- loan_amnt: float (nullable = true)
 |-- term: integer (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_status_binary: integer (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- open_acc: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: float (nullable = true)
 |-- fico_range_low: float (nullable = true)
 |-- fico_range_high: float (nullable = true)
 |-- addr_state: string (nullable =

In [6]:
# Preview data
loans_df.select(
    'loan_amnt', 'int_rate', 'grade', 'annual_inc', 
    'loan_status', 'loan_status_binary', 'fico_avg'
).show(10)

+---------+--------+-----+----------+-----------+------------------+--------+
|loan_amnt|int_rate|grade|annual_inc|loan_status|loan_status_binary|fico_avg|
+---------+--------+-----+----------+-----------+------------------+--------+
|  10000.0|   12.12|    B|   50000.0|Charged Off|                 1|   687.0|
|  30750.0|   12.12|    B|   77000.0| Fully Paid|                 0|    NULL|
|   2650.0|   12.12|    B|   20000.0| Fully Paid|                 0|    NULL|
|   1200.0|   12.12|    B|   13000.0| Fully Paid|                 0|   742.0|
|   2500.0|   14.09|    B|   40000.0| Fully Paid|                 0|   682.0|
|   4900.0|     7.9|    A|   28000.0| Fully Paid|                 0|   727.0|
|  15000.0|   12.12|    B|   95000.0| Fully Paid|                 0|    NULL|
|  10000.0|   12.12|    B|   30000.0| Fully Paid|                 0|   697.0|
|   5000.0|   14.33|    C|   21000.0| Fully Paid|                 0|   672.0|
|  18000.0|   18.49|    D|  110000.0|Charged Off|               

In [7]:
# Register as SQL temp view for queries
loans_df.createOrReplaceTempView("loans")
rejected_df.createOrReplaceTempView("rejected")

print("Temp views created: 'loans', 'rejected'")

Temp views created: 'loans', 'rejected'


---
# Part A: Business Analytics with Spark SQL

This section demonstrates SQL capabilities for business intelligence and reporting.

## 3. Exploratory Analytics

In [8]:
# Basic statistics
spark.sql("""
    SELECT 
        COUNT(*) as total_loans,
        ROUND(SUM(loan_amnt), 2) as total_funded,
        ROUND(AVG(loan_amnt), 2) as avg_loan_amount,
        ROUND(AVG(int_rate), 2) as avg_interest_rate,
        ROUND(AVG(annual_inc), 2) as avg_annual_income,
        ROUND(AVG(fico_avg), 0) as avg_fico_score
    FROM loans
""").show()

+-----------+---------------+---------------+-----------------+-----------------+--------------+
|total_loans|   total_funded|avg_loan_amount|avg_interest_rate|avg_annual_income|avg_fico_score|
+-----------+---------------+---------------+-----------------+-----------------+--------------+
|    2231965|3.3501844975E10|       15010.02|             13.1|         77601.46|         700.0|
+-----------+---------------+---------------+-----------------+-----------------+--------------+



In [9]:
# Loan status distribution
spark.sql("""
    SELECT 
        loan_status,
        loan_status_binary,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM loans
    GROUP BY loan_status, loan_status_binary
    ORDER BY count DESC
""").show(truncate=False)

+---------------------------------------------------+------------------+-------+----------+
|loan_status                                        |loan_status_binary|count  |percentage|
+---------------------------------------------------+------------------+-------+----------+
|Fully Paid                                         |0                 |1059591|47.47     |
|Current                                            |0                 |869764 |38.97     |
|Charged Off                                        |1                 |265946 |11.92     |
|Late (31-120 days)                                 |1                 |21326  |0.96      |
|In Grace Period                                    |0                 |8369   |0.37      |
|Late (16-30 days)                                  |1                 |4325   |0.19      |
|Does not meet the credit policy. Status:Fully Paid |0                 |1873   |0.08      |
|Does not meet the credit policy. Status:Charged Off|1                 |732    |

## 4. Default Rate Analysis by Grade

In [10]:
# Default rate by grade - KEY BUSINESS METRIC
default_by_grade = spark.sql("""
    SELECT 
        grade,
        COUNT(*) as total_loans,
        SUM(loan_status_binary) as defaults,
        ROUND(SUM(loan_status_binary) * 100.0 / COUNT(*), 2) as default_rate,
        ROUND(AVG(int_rate), 2) as avg_interest_rate,
        ROUND(AVG(loan_amnt), 2) as avg_loan_amount,
        ROUND(SUM(loan_amnt), 2) as total_funded
    FROM loans
    GROUP BY grade
    ORDER BY grade
""")

default_by_grade.show()

+-----+-----------+--------+------------+-----------------+---------------+-------------+
|grade|total_loans|defaults|default_rate|avg_interest_rate|avg_loan_amount| total_funded|
+-----+-----------+--------+------------+-----------------+---------------+-------------+
|    A|     426139|   15691|        3.68|             7.08|       14559.68| 6.20444575E9|
|    B|     654756|   57843|        8.83|            10.67|       14135.15|9.255076375E9|
|    C|     642765|   93971|       14.62|            14.14|       15003.03|9.643423525E9|
|    D|     320918|   66389|       20.69|            18.14|       15679.36|  5.0317904E9|
|    E|     134102|   38417|       28.65|            21.83|       17412.39| 2.33503685E9|
|    F|      41272|   15200|       36.83|            25.47|       19081.31|   7.875238E8|
|    G|      12013|    4857|       40.43|            28.11|       20356.97| 2.44548275E8|
+-----+-----------+--------+------------+-----------------+---------------+-------------+



In [11]:
# Save as Gold table
default_by_grade.write.mode("overwrite").parquet(f"{GOLD_PATH}/default_rate_by_grade")
print(f"Saved: {GOLD_PATH}/default_rate_by_grade")

Saved: ../data/medallion/gold//default_rate_by_grade


## 5. Default Rate Analysis by Sub-Grade

In [12]:
# Default rate by sub-grade - more granular view
default_by_subgrade = spark.sql("""
    SELECT 
        grade,
        sub_grade,
        COUNT(*) as total_loans,
        SUM(loan_status_binary) as defaults,
        ROUND(SUM(loan_status_binary) * 100.0 / COUNT(*), 2) as default_rate,
        ROUND(AVG(int_rate), 2) as avg_interest_rate,
        ROUND(AVG(fico_avg), 0) as avg_fico
    FROM loans
    GROUP BY grade, sub_grade
    ORDER BY grade, sub_grade
""")

default_by_subgrade.show(25)

+-----+---------+-----------+--------+------------+-----------------+--------+
|grade|sub_grade|total_loans|defaults|default_rate|avg_interest_rate|avg_fico|
+-----+---------+-----------+--------+------------+-----------------+--------+
|    A|       A1|      85215|    1580|        1.85|              5.6|   747.0|
|    A|       A2|      68445|    1950|        2.85|             6.55|   741.0|
|    A|       A3|      72075|    2359|        3.27|             7.09|   730.0|
|    A|       A4|      94388|    3976|        4.21|             7.56|   725.0|
|    A|       A5|     106016|    5826|        5.50|             8.19|   716.0|
|    B|       B1|     123615|    8207|        6.64|             9.08|   708.0|
|    B|       B2|     124895|    9251|        7.41|             9.97|   705.0|
|    B|       B3|     129649|   11552|        8.91|             10.7|   701.0|
|    B|       B4|     137960|   13549|        9.82|            11.37|   699.0|
|    B|       B5|     138637|   15284|       11.02| 

In [13]:
# Save
default_by_subgrade.write.mode("overwrite").parquet(f"{GOLD_PATH}/default_rate_by_subgrade")
print(f"Saved: {GOLD_PATH}/default_rate_by_subgrade")

Saved: ../data/medallion/gold//default_rate_by_subgrade


## 6. Geographic Analysis

In [14]:
# Default rate by state - TOP 10 states by loan volume
state_analysis = spark.sql("""
    SELECT 
        addr_state as state,
        COUNT(*) as total_loans,
        ROUND(SUM(loan_amnt), 2) as total_funded,
        SUM(loan_status_binary) as defaults,
        ROUND(SUM(loan_status_binary) * 100.0 / COUNT(*), 2) as default_rate,
        ROUND(AVG(annual_inc), 2) as avg_income,
        ROUND(AVG(fico_avg), 0) as avg_fico
    FROM loans
    WHERE addr_state IS NOT NULL
    GROUP BY addr_state
    ORDER BY total_loans DESC
    LIMIT 15
""")

state_analysis.show(15)

+-----+-----------+-------------+--------+------------+----------+--------+
|state|total_loans| total_funded|defaults|default_rate|avg_income|avg_fico|
+-----+-----------+-------------+--------+------------+----------+--------+
|   CA|     303285| 4.63607205E9|   40821|       13.46|  83594.96|   700.0|
|   TX|     181279| 2.84866395E9|   23514|       12.97|  82596.21|   701.0|
|   NY|     180893| 2.68261715E9|   25989|       14.37|  80752.71|   701.0|
|   FL|     157442|2.266215575E9|   22074|       14.02|  73001.78|   699.0|
|   IL|      88439|1.366682175E9|    9935|       11.23|  79607.03|   701.0|
|   NJ|      80543|   1.273376E9|   10843|       13.46|  88319.72|   701.0|
|   PA|      74607|  1.0963789E9|   10073|       13.50|  73704.66|   701.0|
|   OH|      73026| 1.04453825E9|    9411|       12.89|  69120.56|   700.0|
|   GA|      71927|1.100590825E9|    8559|       11.90|  77571.05|   699.0|
|   NC|      61021|   9.036662E8|    8374|       13.72|  73637.53|   701.0|
|   VA|     

In [15]:
# Full state analysis for Gold layer
full_state_analysis = spark.sql("""
    SELECT 
        addr_state as state,
        COUNT(*) as total_loans,
        ROUND(SUM(loan_amnt), 2) as total_funded,
        SUM(loan_status_binary) as defaults,
        ROUND(SUM(loan_status_binary) * 100.0 / COUNT(*), 2) as default_rate,
        ROUND(AVG(annual_inc), 2) as avg_income,
        ROUND(AVG(int_rate), 2) as avg_interest_rate,
        ROUND(AVG(fico_avg), 0) as avg_fico
    FROM loans
    WHERE addr_state IS NOT NULL
    GROUP BY addr_state
    ORDER BY total_loans DESC
""")

full_state_analysis.write.mode("overwrite").parquet(f"{GOLD_PATH}/loan_analysis_by_state")
print(f"Saved: {GOLD_PATH}/loan_analysis_by_state")

Saved: ../data/medallion/gold//loan_analysis_by_state


## 7. Loan Purpose Analysis

In [16]:
# Default rate by loan purpose
purpose_analysis = spark.sql("""
    SELECT 
        purpose,
        COUNT(*) as total_loans,
        ROUND(SUM(loan_amnt), 2) as total_funded,
        SUM(loan_status_binary) as defaults,
        ROUND(SUM(loan_status_binary) * 100.0 / COUNT(*), 2) as default_rate,
        ROUND(AVG(loan_amnt), 2) as avg_loan_amount,
        ROUND(AVG(int_rate), 2) as avg_interest_rate
    FROM loans
    WHERE purpose IS NOT NULL
    GROUP BY purpose
    ORDER BY total_loans DESC
""")

purpose_analysis.show(15, truncate=False)

+------------------+-----------+--------------+--------+------------+---------------+-----------------+
|purpose           |total_loans|total_funded  |defaults|default_rate|avg_loan_amount|avg_interest_rate|
+------------------+-----------+--------------+--------+------------+---------------+-----------------+
|debt_consolidation|1241832    |1.980251725E10|175366  |14.12       |15946.21       |13.53            |
|credit_card       |503099     |7.69429535E9  |53532   |10.64       |15293.8        |11.69            |
|home_improvement  |145791     |2.13439705E9  |16835   |11.55       |14640.12       |12.64            |
|other             |135726     |1.421758975E9 |17863   |13.16       |10475.21       |14.25            |
|major_purchase    |48778      |6.21745575E8  |6008    |12.32       |12746.43       |12.8             |
|medical           |26841      |2.538256E8    |3666    |13.66       |9456.64        |13.65            |
|car               |23000      |2.17284725E8  |2286    |9.94    

In [17]:
purpose_analysis.write.mode("overwrite").parquet(f"{GOLD_PATH}/loan_analysis_by_purpose")
print(f"Saved: {GOLD_PATH}/loan_analysis_by_purpose")

Saved: ../data/medallion/gold//loan_analysis_by_purpose


## 8. Time Series Analysis

In [18]:
# Loan trends over time (by issue year)
time_analysis = spark.sql("""
    SELECT 
        SUBSTRING(issue_d, 1, 4) as year,
        COUNT(*) as total_loans,
        ROUND(SUM(loan_amnt) / 1000000, 2) as total_funded_millions,
        SUM(loan_status_binary) as defaults,
        ROUND(SUM(loan_status_binary) * 100.0 / COUNT(*), 2) as default_rate,
        ROUND(AVG(loan_amnt), 2) as avg_loan_amount,
        ROUND(AVG(int_rate), 2) as avg_interest_rate
    FROM loans
    WHERE issue_d IS NOT NULL
    GROUP BY SUBSTRING(issue_d, 1, 4)
    ORDER BY year
""")

time_analysis.show(15)

+----+-----------+---------------------+--------+------------+---------------+-----------------+
|year|total_loans|total_funded_millions|defaults|default_rate|avg_loan_amount|avg_interest_rate|
+----+-----------+---------------------+--------+------------+---------------+-----------------+
|2007|        556|                  4.5|     148|       26.62|        8087.37|            11.76|
|2008|       2268|                19.91|     470|       20.72|        8780.04|            12.05|
|2009|       4977|                48.63|     693|       13.92|        9769.96|            12.44|
|2010|      11964|               125.53|    1696|       14.18|       10492.67|            12.01|
|2011|      20540|               246.84|    3163|       15.40|       12017.58|            12.25|
|2012|      50678|               679.77|    8315|       16.41|       13413.47|            13.65|
|2013|     130064|              1906.55|   20412|       15.69|       14658.54|            14.54|
|2014|     233196|            

In [19]:
time_analysis.write.mode("overwrite").parquet(f"{GOLD_PATH}/loan_trends_by_year")
print(f"Saved: {GOLD_PATH}/loan_trends_by_year")

Saved: ../data/medallion/gold//loan_trends_by_year


## 9. Risk Segmentation Analysis

In [20]:
# Create risk segments based on FICO score
risk_segments = spark.sql("""
    SELECT 
        CASE 
            WHEN fico_avg >= 750 THEN 'Excellent (750+)'
            WHEN fico_avg >= 700 THEN 'Good (700-749)'
            WHEN fico_avg >= 650 THEN 'Fair (650-699)'
            WHEN fico_avg >= 600 THEN 'Poor (600-649)'
            ELSE 'Very Poor (<600)'
        END as fico_segment,
        COUNT(*) as total_loans,
        SUM(loan_status_binary) as defaults,
        ROUND(SUM(loan_status_binary) * 100.0 / COUNT(*), 2) as default_rate,
        ROUND(AVG(loan_amnt), 2) as avg_loan_amount,
        ROUND(AVG(int_rate), 2) as avg_interest_rate,
        ROUND(AVG(annual_inc), 2) as avg_income,
        ROUND(AVG(dti), 2) as avg_dti
    FROM loans
    WHERE fico_avg IS NOT NULL
    GROUP BY 
        CASE 
            WHEN fico_avg >= 750 THEN 'Excellent (750+)'
            WHEN fico_avg >= 700 THEN 'Good (700-749)'
            WHEN fico_avg >= 650 THEN 'Fair (650-699)'
            WHEN fico_avg >= 600 THEN 'Poor (600-649)'
            ELSE 'Very Poor (<600)'
        END
    ORDER BY default_rate
""")

risk_segments.show(truncate=False)

+----------------+-----------+--------+------------+---------------+-----------------+----------+-------+
|fico_segment    |total_loans|defaults|default_rate|avg_loan_amount|avg_interest_rate|avg_income|avg_dti|
+----------------+-----------+--------+------------+---------------+-----------------+----------+-------+
|Excellent (750+)|199513     |10490   |5.26        |16083.43       |8.85             |83875.33  |16.78  |
|Good (700-749)  |723495     |70562   |9.75        |16329.0        |11.73            |81770.08  |19.47  |
|Fair (650-699)  |1271154    |205475  |16.16       |14129.09       |14.54            |74425.5   |18.92  |
|Very Poor (<600)|493        |88      |17.85       |14048.17       |14.94            |76527.56  |NULL   |
|Poor (600-649)  |108        |34      |31.48       |6280.09        |15.41            |51071.57  |14.07  |
+----------------+-----------+--------+------------+---------------+-----------------+----------+-------+



In [21]:
risk_segments.write.mode("overwrite").parquet(f"{GOLD_PATH}/risk_segments_by_fico")
print(f"Saved: {GOLD_PATH}/risk_segments_by_fico")

Saved: ../data/medallion/gold//risk_segments_by_fico


## 10. Advanced SQL: Window Functions

In [22]:
# Rank states by default rate within each grade
spark.sql("""
    WITH state_grade_stats AS (
        SELECT 
            grade,
            addr_state as state,
            COUNT(*) as loan_count,
            ROUND(SUM(loan_status_binary) * 100.0 / COUNT(*), 2) as default_rate
        FROM loans
        WHERE addr_state IS NOT NULL
        GROUP BY grade, addr_state
        HAVING COUNT(*) >= 100  -- Filter for statistical significance
    )
    SELECT 
        grade,
        state,
        loan_count,
        default_rate,
        RANK() OVER (PARTITION BY grade ORDER BY default_rate DESC) as risk_rank
    FROM state_grade_stats
    ORDER BY grade, risk_rank
""").show(30)

+-----+------------------+----------+------------+---------+
|grade|             state|loan_count|default_rate|risk_rank|
+-----+------------------+----------+------------+---------+
|    A|Debt Consolidation|       101|        8.91|        1|
|    A|Debt consolidation|       107|        6.54|        2|
|    A|                NM|      2175|        4.92|        3|
|    A|                MS|      2059|        4.71|        4|
|    A|debt_consolidation|       328|        4.57|        5|
|    A|                OK|      3605|        4.47|        6|
|    A|                LA|      4716|        4.35|        7|
|    A|                AR|      2855|        4.34|        8|
|    A|                SD|       763|        4.33|        9|
|    A|                NY|     32822|        4.28|       10|
|    A|                AL|      4348|        4.28|       10|
|    A|                NJ|     15950|        4.19|       12|
|    A|                NV|      5677|        4.07|       13|
|    A|                F

---
# Part B: Machine Learning with MLlib

This section builds a loan default prediction model using Spark MLlib.

## 11. Prepare Data for ML

In [23]:
# Select features for the model
# Numeric features
numeric_features = [
    'loan_amnt',
    'int_rate',
    'installment',
    'annual_inc',
    'dti',
    'open_acc',
    'pub_rec',
    'revol_bal',
    'revol_util',
    'total_acc',
    'fico_avg',
    'loan_to_income',
    'delinq_2yrs',
    'inq_last_6mths'
]

# Categorical features
categorical_features = [
    'term',
    'grade',
    'home_ownership',
    'verification_status',
    'purpose'
]

# Target variable
target = 'loan_status_binary'

print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")
print(f"Target: {target}")

Numeric features: 14
Categorical features: 5
Target: loan_status_binary


In [24]:
# Select columns and filter nulls in target
all_features = numeric_features + categorical_features + [target]

ml_df = loans_df.select(all_features).filter(F.col(target).isNotNull())

print(f"ML Dataset: {ml_df.count():,} rows")
ml_df.show(5)

ML Dataset: 2,231,965 rows
+---------+--------+-----------+----------+-----+--------+-------+---------+----------+---------+--------+--------------+-----------+--------------+----+-----+--------------+-------------------+--------------------+------------------+
|loan_amnt|int_rate|installment|annual_inc|  dti|open_acc|pub_rec|revol_bal|revol_util|total_acc|fico_avg|loan_to_income|delinq_2yrs|inq_last_6mths|term|grade|home_ownership|verification_status|             purpose|loan_status_binary|
+---------+--------+-----------+----------+-----+--------+-------+---------+----------+---------+--------+--------------+-----------+--------------+----+-----+--------------+-------------------+--------------------+------------------+
|  10000.0|   12.12|     332.72|   50000.0|17.71|    21.0|    0.0|  24820.0|      42.8|     39.0|   687.0|           0.2|        1.0|           0.0|  36|    B|      MORTGAGE|       Not Verified|         credit_card|                 1|
|  30750.0|   12.12|    1023.11| 

In [25]:
# # --- FIX FOR EMPTY STRINGS ---
# # The Silver layer accidentally allowed empty strings "" (from whitespace).
# # We must convert these to "Unknown" or None before ML processing.

# print("Sanitizing empty strings in categorical features...")

# for col_name in categorical_features:
#     # Replace empty strings or whitespace-only strings with "Unknown"
#     ml_df = ml_df.withColumn(
#         col_name, 
#         F.when(F.trim(F.col(col_name)) == "", "Unknown")
#         .otherwise(F.col(col_name))
#     )

# # Also ensure no nulls exist (filling them just in case)
# ml_df = ml_df.fillna("Unknown", subset=categorical_features)

# print("Sanitization complete. Empty strings replaced with 'Unknown'.")

In [26]:
# Check class distribution (important for imbalanced data)
ml_df.groupBy(target).count().show()

+------------------+-------+
|loan_status_binary|  count|
+------------------+-------+
|                 1| 292368|
|                 0|1939597|
+------------------+-------+



In [27]:
# Check for nulls in features
print("Null counts per column:")
null_counts = ml_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in ml_df.columns])
null_counts.show(truncate=False)

Null counts per column:


+---------+--------+-----------+----------+-----+--------+-------+---------+----------+---------+--------+--------------+-----------+--------------+----+-----+--------------+-------------------+-------+------------------+
|loan_amnt|int_rate|installment|annual_inc|dti  |open_acc|pub_rec|revol_bal|revol_util|total_acc|fico_avg|loan_to_income|delinq_2yrs|inq_last_6mths|term|grade|home_ownership|verification_status|purpose|loan_status_binary|
+---------+--------+-----------+----------+-----+--------+-------+---------+----------+---------+--------+--------------+-----------+--------------+----+-----+--------------+-------------------+-------+------------------+
|0        |0       |0          |0         |37711|24274   |12047  |6342     |5611      |2548     |37202   |0             |19209      |7148          |0   |0    |49            |0                  |43     |0                 |
+---------+--------+-----------+----------+-----+--------+-------+---------+----------+---------+--------+------

## 12. Feature Engineering Pipeline

In [28]:
# Handle missing values in numeric columns
# Fill with median using Imputer
imputer = Imputer(
    inputCols=numeric_features,
    outputCols=[f"{c}_imputed" for c in numeric_features],
    strategy="median"
)

imputed_numeric_features = [f"{c}_imputed" for c in numeric_features]
print(f"Imputer configured for {len(numeric_features)} numeric features")

Imputer configured for 14 numeric features


In [29]:
# String Indexers for categorical features
# This converts strings to numeric indices

indexers = []
indexed_cat_features = []

for cat_col in categorical_features:
    indexer = StringIndexer(
        inputCol=cat_col, 
        outputCol=f"{cat_col}_indexed",
        handleInvalid="keep"  # Handle unseen labels
    )
    indexers.append(indexer)
    indexed_cat_features.append(f"{cat_col}_indexed")

print(f"Created {len(indexers)} StringIndexers")

Created 5 StringIndexers


In [30]:
# One-Hot Encoders for categorical features
encoders = []
encoded_cat_features = []

for cat_col in categorical_features:
    encoder = OneHotEncoder(
        inputCol=f"{cat_col}_indexed",
        outputCol=f"{cat_col}_encoded",
        handleInvalid="keep"
    )
    encoders.append(encoder)
    encoded_cat_features.append(f"{cat_col}_encoded")

print(f"Created {len(encoders)} OneHotEncoders")

Created 5 OneHotEncoders


In [31]:
# Assemble all features into a single vector
all_feature_cols = imputed_numeric_features + encoded_cat_features

assembler = VectorAssembler(
    inputCols=all_feature_cols,
    outputCol="features_unscaled",
    handleInvalid="skip"  # Skip rows with nulls
)

print(f"VectorAssembler will combine {len(all_feature_cols)} feature columns")

VectorAssembler will combine 19 feature columns


In [32]:
# Scale features (important for Logistic Regression)
scaler = StandardScaler(
    inputCol="features_unscaled",
    outputCol="features",
    withStd=True,
    withMean=False  # Don't center for sparse data
)

print("StandardScaler configured")

StandardScaler configured


## 13. Train/Test Split

In [33]:
# Split data into training and test sets
train_df, test_df = ml_df.randomSplit([0.8, 0.2], seed=42)

# Cache for performance
train_df.cache()
test_df.cache()

train_count = train_df.count()
test_count = test_df.count()

print(f"Training set: {train_count:,} rows ({train_count/(train_count+test_count)*100:.1f}%)")
print(f"Test set: {test_count:,} rows ({test_count/(train_count+test_count)*100:.1f}%)")

Training set: 1,785,954 rows (80.0%)
Test set: 446,011 rows (20.0%)


In [34]:
# Verify class distribution in splits
print("Training set class distribution:")
train_df.groupBy(target).count().show()

print("Test set class distribution:")
test_df.groupBy(target).count().show()

Training set class distribution:
+------------------+-------+
|loan_status_binary|  count|
+------------------+-------+
|                 1| 234024|
|                 0|1551930|
+------------------+-------+

Test set class distribution:
+------------------+------+
|loan_status_binary| count|
+------------------+------+
|                 1| 58344|
|                 0|387667|
+------------------+------+



## 14. Model 1: Logistic Regression

In [35]:
# Create Logistic Regression model
lr = LogisticRegression(
    featuresCol="features",
    labelCol=target,
    maxIter=100,
    regParam=0.01,
    elasticNetParam=0.8  # L1/L2 mix
)

# Build pipeline
lr_pipeline = Pipeline(stages=[
    imputer,
    *indexers,
    *encoders,
    assembler,
    scaler,
    lr
])

print(f"Logistic Regression Pipeline: {len(lr_pipeline.getStages())} stages")

Logistic Regression Pipeline: 14 stages


In [36]:
%%time
# Train the model
print("Training Logistic Regression model...")
lr_model = lr_pipeline.fit(train_df)
print("Training complete!")

Training Logistic Regression model...


Training complete!
CPU times: user 96.6 ms, sys: 47 ms, total: 144 ms
Wall time: 36.4 s


In [37]:
# Make predictions
lr_predictions = lr_model.transform(test_df)

# Show sample predictions
lr_predictions.select(
    'loan_amnt', 'int_rate', 'grade', 'fico_avg',
    target, 'prediction', 'probability'
).show(10, truncate=False)

+---------+--------+-----+--------+------------------+----------+-----------------------------------------+
|loan_amnt|int_rate|grade|fico_avg|loan_status_binary|prediction|probability                              |
+---------+--------+-----+--------+------------------+----------+-----------------------------------------+
|1000.0   |5.32    |A    |742.0   |0                 |0.0       |[0.9590079632618337,0.040992036738166315]|
|1000.0   |5.32    |A    |777.0   |0                 |0.0       |[0.9531010349517048,0.04689896504829516] |
|1000.0   |5.32    |A    |802.0   |0                 |0.0       |[0.964517878394326,0.03548212160567399]  |
|1000.0   |6.08    |A    |747.0   |0                 |0.0       |[0.9532996313017876,0.04670036869821237] |
|1000.0   |6.72    |A    |787.0   |0                 |0.0       |[0.9578190301664191,0.042180969833580884]|
|1000.0   |6.97    |A    |757.0   |0                 |0.0       |[0.9538840366282096,0.04611596337179036] |
|1000.0   |7.07    |A    |79

In [38]:
# Evaluate Logistic Regression
evaluator_auc = BinaryClassificationEvaluator(
    labelCol=target,
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

evaluator_pr = BinaryClassificationEvaluator(
    labelCol=target,
    rawPredictionCol="rawPrediction",
    metricName="areaUnderPR"
)

evaluator_accuracy = MulticlassClassificationEvaluator(
    labelCol=target,
    predictionCol="prediction",
    metricName="accuracy"
)

lr_auc = evaluator_auc.evaluate(lr_predictions)
lr_pr = evaluator_pr.evaluate(lr_predictions)
lr_accuracy = evaluator_accuracy.evaluate(lr_predictions)

print("=" * 50)
print("LOGISTIC REGRESSION RESULTS")
print("=" * 50)
print(f"AUC-ROC: {lr_auc:.4f}")
print(f"AUC-PR:  {lr_pr:.4f}")
print(f"Accuracy: {lr_accuracy:.4f}")

LOGISTIC REGRESSION RESULTS
AUC-ROC: 0.6878
AUC-PR:  0.2305
Accuracy: 0.8691


In [39]:
# Confusion Matrix
print("\nConfusion Matrix:")
lr_predictions.groupBy(target, 'prediction').count().orderBy(target, 'prediction').show()


Confusion Matrix:


+------------------+----------+------+
|loan_status_binary|prediction| count|
+------------------+----------+------+
|                 0|       0.0|387639|
|                 0|       1.0|    28|
|                 1|       0.0| 58337|
|                 1|       1.0|     7|
+------------------+----------+------+



## 15. Model 2: Random Forest

In [40]:
# Create Random Forest model (doesn't need scaling)
rf = RandomForestClassifier(
    featuresCol="features_unscaled",  # Use unscaled features
    labelCol=target,
    numTrees=20,
    maxDepth=5,
    minInstancesPerNode=100,
    seed=42
)

# Build pipeline (without scaler)
rf_pipeline = Pipeline(stages=[
    imputer,
    *indexers,
    *encoders,
    assembler,
    rf
])

print(f"Random Forest Pipeline: {len(rf_pipeline.getStages())} stages")

Random Forest Pipeline: 13 stages


In [41]:
%%time
# Train the model
print("Training Random Forest model...")
rf_model = rf_pipeline.fit(train_df)
print("Training complete!")

Training Random Forest model...


25/11/27 12:46:24 ERROR TaskSchedulerImpl: Lost executor 1 on 192.168.18.110: Command exited with code 137
25/11/27 12:47:15 ERROR TaskSchedulerImpl: Lost executor 4 on 192.168.18.110: Command exited with code 137


Training complete!
CPU times: user 365 ms, sys: 914 ms, total: 1.28 s
Wall time: 3min 15s


In [42]:
# Make predictions
rf_predictions = rf_model.transform(test_df)

# Evaluate Random Forest
rf_auc = evaluator_auc.evaluate(rf_predictions)
rf_pr = evaluator_pr.evaluate(rf_predictions)
rf_accuracy = evaluator_accuracy.evaluate(rf_predictions)

print("=" * 50)
print("RANDOM FOREST RESULTS")
print("=" * 50)
print(f"AUC-ROC: {rf_auc:.4f}")
print(f"AUC-PR:  {rf_pr:.4f}")
print(f"Accuracy: {rf_accuracy:.4f}")

RANDOM FOREST RESULTS
AUC-ROC: 0.5000
AUC-PR:  0.1308
Accuracy: 0.8692


In [43]:
# Confusion Matrix
print("\nConfusion Matrix:")
rf_predictions.groupBy(target, 'prediction').count().orderBy(target, 'prediction').show()


Confusion Matrix:


+------------------+----------+------+
|loan_status_binary|prediction| count|
+------------------+----------+------+
|                 0|       0.0|387667|
|                 1|       0.0| 58344|
+------------------+----------+------+



In [44]:
# Feature Importance (Random Forest)
rf_model_final = rf_model.stages[-1]
feature_importance = rf_model_final.featureImportances

# Get feature names
# Note: This is an approximation since OneHot expands features
print("\nTop Feature Importances (by index):")
importance_list = [(i, float(imp)) for i, imp in enumerate(feature_importance)]
sorted_importance = sorted(importance_list, key=lambda x: x[1], reverse=True)[:15]

print(f"{'Index':<8} {'Importance':<12}")
print("-" * 20)
for idx, imp in sorted_importance:
    print(f"{idx:<8} {imp:.4f}")


Top Feature Importances (by index):
Index    Importance  
--------------------
0        0.0000
1        0.0000
2        0.0000
3        0.0000
4        0.0000
5        0.0000
6        0.0000
7        0.0000
8        0.0000
9        0.0000
10       0.0000
11       0.0000
12       0.0000
13       0.0000
14       0.0000


## 16. Model Comparison

In [45]:
# Compare models
print("=" * 60)
print("MODEL COMPARISON")
print("=" * 60)
print(f"{'Metric':<15} {'Logistic Regression':<20} {'Random Forest':<20}")
print("-" * 60)
print(f"{'AUC-ROC':<15} {lr_auc:<20.4f} {rf_auc:<20.4f}")
print(f"{'AUC-PR':<15} {lr_pr:<20.4f} {rf_pr:<20.4f}")
print(f"{'Accuracy':<15} {lr_accuracy:<20.4f} {rf_accuracy:<20.4f}")
print("=" * 60)

# Determine best model
best_model_name = "Random Forest" if rf_auc > lr_auc else "Logistic Regression"
best_model = rf_model if rf_auc > lr_auc else lr_model
best_auc = max(rf_auc, lr_auc)

print(f"\nBest Model: {best_model_name} (AUC: {best_auc:.4f})")

MODEL COMPARISON
Metric          Logistic Regression  Random Forest       
------------------------------------------------------------
AUC-ROC         0.6878               0.5000              
AUC-PR          0.2305               0.1308              
Accuracy        0.8691               0.8692              

Best Model: Logistic Regression (AUC: 0.6878)


## 17. Hyperparameter Tuning with Cross-Validation (Optional)

In [46]:
# Hyperparameter tuning for Random Forest
# Note: This can be time-consuming, so we use a small grid

# Create a new RF for tuning
rf_tune = RandomForestClassifier(
    featuresCol="features_unscaled",
    labelCol=target,
    seed=42
)

# Pipeline for tuning
rf_tune_pipeline = Pipeline(stages=[
    imputer,
    *indexers,
    *encoders,
    assembler,
    rf_tune
])

# Parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(rf_tune.numTrees, [50, 100]) \
    .addGrid(rf_tune.maxDepth, [5, 10]) \
    .build()

print(f"Parameter grid size: {len(paramGrid)} combinations")

Parameter grid size: 4 combinations


In [47]:
%%time
# Cross-validator
cv = CrossValidator(
    estimator=rf_tune_pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator_auc,
    numFolds=3,
    seed=42
)

# Sample data for faster tuning (optional - remove for full tuning)
train_sample = train_df.sample(0.2, seed=42)
print(f"Training CV on sample of {train_sample.count():,} rows...")

# Fit cross-validator
cv_model = cv.fit(train_sample)
print("Cross-validation complete!")

Training CV on sample of 357,747 rows...


Cross-validation complete!
CPU times: user 2.41 s, sys: 1.34 s, total: 3.75 s
Wall time: 5min 20s


In [48]:
# Best model from CV
print("Cross-validation AUC scores:")
for i, score in enumerate(cv_model.avgMetrics):
    print(f"  Config {i+1}: {score:.4f}")

print(f"\nBest CV AUC: {max(cv_model.avgMetrics):.4f}")

Cross-validation AUC scores:
  Config 1: 0.6914
  Config 2: 0.6945
  Config 3: 0.6915
  Config 4: 0.6942

Best CV AUC: 0.6945


In [49]:
# Evaluate best CV model on test set
cv_predictions = cv_model.transform(test_df)
cv_auc = evaluator_auc.evaluate(cv_predictions)
print(f"Best CV Model Test AUC: {cv_auc:.4f}")

Best CV Model Test AUC: 0.6924


## 18. Save Models and Predictions

In [50]:
# Save best model
MODEL_PATH = f"{GOLD_PATH}/models/default_prediction_model"
best_model.write().overwrite().save(MODEL_PATH)
print(f"Model saved to: {MODEL_PATH}")

Model saved to: ../data/medallion/gold//models/default_prediction_model


In [51]:
# Save predictions
# Use the predictions from the best model
best_predictions = rf_predictions if rf_auc > lr_auc else lr_predictions

predictions_to_save = best_predictions.select(
    'loan_amnt', 'int_rate', 'term', 'grade',
    'annual_inc', 'dti', 'fico_avg', 'purpose', 'home_ownership',
    target, 'prediction', 'probability'
)

PREDICTIONS_PATH = f"{GOLD_PATH}/predictions"
predictions_to_save.write.mode("overwrite").parquet(PREDICTIONS_PATH)
print(f"Predictions saved to: {PREDICTIONS_PATH}")

Predictions saved to: ../data/medallion/gold//predictions


## 19. Create Risk Scoring Table

In [52]:
# Create a risk scoring summary for business use
# Extract probability of default from the probability vector

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# UDF to extract probability of default (class 1)
@udf(FloatType())
def extract_prob_default(probability):
    return float(probability[1])

risk_scores = best_predictions.withColumn(
    "default_probability", 
    extract_prob_default(F.col("probability"))
)

# Create risk categories
risk_scores = risk_scores.withColumn(
    "risk_category",
    F.when(F.col("default_probability") < 0.1, "Low Risk")
    .when(F.col("default_probability") < 0.25, "Medium Risk")
    .when(F.col("default_probability") < 0.5, "High Risk")
    .otherwise("Very High Risk")
)

# Show distribution
risk_scores.groupBy("risk_category").agg(
    F.count("*").alias("count"),
    F.round(F.avg("default_probability"), 4).alias("avg_prob"),
    F.round(F.avg(target), 4).alias("actual_default_rate")
).orderBy("avg_prob").show()

+--------------+------+--------+-------------------+
| risk_category| count|avg_prob|actual_default_rate|
+--------------+------+--------+-------------------+
|      Low Risk|147438|  0.0686|             0.0522|
|   Medium Risk|274452|  0.1498|              0.158|
|     High Risk| 24086|  0.3012|             0.3024|
|Very High Risk|    35|  0.5969|                0.2|
+--------------+------+--------+-------------------+



In [53]:
# Save risk scores
risk_score_table = risk_scores.select(
    'loan_amnt', 'int_rate', 'grade', 'fico_avg', 'annual_inc',
    'purpose', target, 'default_probability', 'risk_category'
)

RISK_SCORES_PATH = f"{GOLD_PATH}/risk_scores"
risk_score_table.write.mode("overwrite").parquet(RISK_SCORES_PATH)
print(f"Risk scores saved to: {RISK_SCORES_PATH}")

Risk scores saved to: ../data/medallion/gold//risk_scores


## 20. Gold Layer Summary

In [54]:
# List all Gold layer outputs
!ls -la {GOLD_PATH}

total 44
drwxrwxr-x 11 ubuntu ubuntu 4096 Nov 27 12:54 .
drwxrwxr-x  5 ubuntu ubuntu 4096 Nov 26 16:25 ..
drwxr-xr-x  2 ubuntu ubuntu 4096 Nov 27 12:44 default_rate_by_grade
drwxr-xr-x  2 ubuntu ubuntu 4096 Nov 27 12:44 default_rate_by_subgrade
drwxr-xr-x  2 ubuntu ubuntu 4096 Nov 27 12:44 loan_analysis_by_purpose
drwxr-xr-x  2 ubuntu ubuntu 4096 Nov 27 12:44 loan_analysis_by_state
drwxr-xr-x  2 ubuntu ubuntu 4096 Nov 27 12:44 loan_trends_by_year
drwxr-xr-x  3 ubuntu ubuntu 4096 Nov 27 12:54 models
drwxr-xr-x  2 ubuntu ubuntu 4096 Nov 27 12:54 predictions
drwxr-xr-x  2 ubuntu ubuntu 4096 Nov 27 12:54 risk_scores
drwxr-xr-x  2 ubuntu ubuntu 4096 Nov 27 12:44 risk_segments_by_fico


In [55]:
# Final Summary
print("=" * 70)
print("GOLD LAYER SUMMARY")
print("=" * 70)

print("\n--- Part A: SQL Analytics ---")
print("Tables created for business intelligence:")
print(f"  1. default_rate_by_grade     - Default rates by loan grade")
print(f"  2. default_rate_by_subgrade  - Default rates by sub-grade")
print(f"  3. loan_analysis_by_state    - Geographic analysis")
print(f"  4. loan_analysis_by_purpose  - Analysis by loan purpose")
print(f"  5. loan_trends_by_year       - Time series analysis")
print(f"  6. risk_segments_by_fico     - FICO-based risk segments")

print("\n--- Part B: Machine Learning ---")
print("Models trained:")
print(f"  1. Logistic Regression - AUC: {lr_auc:.4f}")
print(f"  2. Random Forest       - AUC: {rf_auc:.4f}")
print(f"\nBest model: {best_model_name}")

print("\nML outputs:")
print(f"  - models/default_prediction_model - Trained ML pipeline")
print(f"  - predictions                     - Test set predictions")
print(f"  - risk_scores                     - Risk scoring table")

print("\n--- Technologies Used ---")
print("  - Spark SQL: Complex queries, aggregations, window functions")
print("  - MLlib: Pipeline, VectorAssembler, StringIndexer, OneHotEncoder")
print("  - MLlib: LogisticRegression, RandomForestClassifier")
print("  - MLlib: CrossValidator, BinaryClassificationEvaluator")

print("\n" + "=" * 70)

GOLD LAYER SUMMARY

--- Part A: SQL Analytics ---
Tables created for business intelligence:
  1. default_rate_by_grade     - Default rates by loan grade
  2. default_rate_by_subgrade  - Default rates by sub-grade
  3. loan_analysis_by_state    - Geographic analysis
  4. loan_analysis_by_purpose  - Analysis by loan purpose
  5. loan_trends_by_year       - Time series analysis
  6. risk_segments_by_fico     - FICO-based risk segments

--- Part B: Machine Learning ---
Models trained:
  1. Logistic Regression - AUC: 0.6878
  2. Random Forest       - AUC: 0.5000

Best model: Logistic Regression

ML outputs:
  - models/default_prediction_model - Trained ML pipeline
  - predictions                     - Test set predictions
  - risk_scores                     - Risk scoring table

--- Technologies Used ---
  - Spark SQL: Complex queries, aggregations, window functions
  - MLlib: Pipeline, VectorAssembler, StringIndexer, OneHotEncoder
  - MLlib: LogisticRegression, RandomForestClassifier
  - M

In [57]:
# Clean up
train_df.unpersist()
test_df.unpersist()

print("Cached DataFrames unpersisted.")
print("\nGold layer complete!")

spark.stop()

Cached DataFrames unpersisted.

Gold layer complete!


---
## Conclusion

The Gold layer provides:

1. **Business Analytics** - Ready-to-use aggregated tables for dashboards
2. **ML Model** - Trained loan default prediction model
3. **Risk Scoring** - Probability-based risk categorization

These outputs can be:
- Connected to BI tools (Tableau, Power BI)
- Served via REST API for applications
- Used for real-time scoring of new loan applications